In [1]:
import os
import re
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

import umap
from kneed import knee_locator

In [55]:
import os
os.getcwd()

'/home/jovyan/work/Group work/fsds-group-work/Project'

In [60]:
import os
from requests import get
from urllib.parse import urlparse

def cache_data(src:str, dest:str) -> str:
    """Downloads and caches a remote file locally.
    
    The function sits between the 'read' step of a pandas or geopandas
    data frame and downloading the file from a remote location. The idea
    is that it will save it locally so that you don't need to remember to
    do so yourself. Subsequent re-reads of the file will return instantly
    rather than downloading the entire file for a second or n-th itme.
    
    Parameters
    ----------
    src : str
        The remote *source* for the file, any valid URL should work.
    dest : str
        The *destination* location to save the downloaded file.
        
    Returns
    -------
    str
        A string representing the local location of the file.
    """
    
    url = urlparse(src) # We assume that this is some kind of valid URL 
    fn  = os.path.split(url.path)[-1] # Extract the filename
    dfn = os.path.join(dest,fn) # Destination filename
    
    # Check if dest+filename does *not* exist -- 
    # that would mean we have to download it!
    if not os.path.isfile(dfn):
        
        print(f"{dfn} not found, downloading!")

        # Convert the path back into a list (without)
        # the filename -- we need to check that directories
        # exist first.
        path = os.path.split(dest)
        
        # Create any missing directories in dest(ination) path
        # -- os.path.join is the reverse of split (as you saw above)
        # but it doesn't work with lists... so I had to google how
        # to use the 'splat' operator! os.makedirs creates missing
        # directories in a path automatically.
        if len(path) >= 1 and path[0] != '':
            os.makedirs(os.path.join(*path), exist_ok=True)
            
        # Download and write the file
        with open(dfn, "wb") as file:
            response = get(src)
            file.write(response.content)
            
        print('Done downloading...')

    else:
        print(f"Found {dfn} locally!")

    return dfn

In [68]:
src_url  = 'https://www.dropbox.com/home/Data/Inside-Airbnb-Data-Dictionary.xlsx'
dest_path = os.path.join('')
df = pd.read_excel(cache_data(src_url, dest_path))

Inside-Airbnb-Data-Dictionary.xlsx not found, downloading!
Done downloading...


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [47]:
import pandas as pd
import requests
from io import StringIO

# 替换为你的OneDrive共享链接
onedrive_link = "https://onedrive.live.com/?id=root&cid=6600E29CD1DB5E0C/test.xlsx"

# 将链接中的"view"替换为"download"
download_link = onedrive_link.replace("view", "download")

# 发送HTTP请求并获取文件内容
response = requests.get(download_link)

# 检查响应状态码
if response.status_code == 200:
    # 使用pandas的read_csv函数读取CSV数据
    df = pd.read_excel(StringIO(response.text))
    # 打印数据框的前几行
    print(df.head())
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [91]:
import dropbox

In [92]:
dbx = dropbox.Dropbox("sl.BrLlVK2NBfLgW_aw_x_ziDLuiImeNoXw6ObKoc2Q2QjoyHorEHFHeu06ILtnMlHkFJUyOhWf2mdAD7Wpz1dLvJzCdmWq-QYVUTN2tR0SabEcHR5RyEcqWRsazQzln5A0GIQCUtAAo9aoq7k8ix8P46I")

In [98]:
metadata, res = dbx.files_download("/Data/msoa-data.xls")

In [101]:
print(res)

<Response [200]>


In [102]:
with open("msoa-data-test.xls", "wb") as f:
    f.write(res.content)

In [103]:
pd.read_excel("msoa-data-test.xls")

,Unnamed: 0,Unnamed: 1,Age Structure (2011 Census),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Mid-year Estimate totals,...,Unnamed: 197,Unnamed: 198,Unnamed: 199,Unnamed: 200,Unnamed: 201,Unnamed: 202,Unnamed: 203,Unnamed: 204,Unnamed: 205,Unnamed: 206
0,NaN,NaN,All Ages,0-15,16-29,30-44,45-64,65+,Working-age,All Ages,...,NaN,NaN,2011,NaN,NaN,NaN,2012,NaN,NaN,NaN
1,MSOA Code,MSOA Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002,...,Slight,2010 Total,Fatal,Serious,Slight,2011 Total,Fatal,Serious,Slight,2012 Total
2,E02000001,City of London 001,7375,620,1665,2045,2010,1035,5720,7280,...,334,374,0,46,359,405,2,51,361,414
3,E02000002,Barking and Dagenham 001,6775,1751,1277,1388,1258,1101,3923,6333,...,18,18,0,2,16,18,0,1,15,16
4,E02000003,Barking and Dagenham 002,10045,2247,1959,2300,2259,1280,6518,9236,...,34,37,1,4,40,45,0,3,47,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,E02000980,Westminster 021,8684,1277,1946,2317,1993,1151,6256,7780,...,23,27,1,5,28,34,0,5,25,30
982,E02000981,Westminster 022,8991,913,2475,2781,1969,853,7225,8745,...,28,32,0,1,21,22,0,2,8,10
983,E02000982,Westminster 023,8226,1446,1689,2154,1835,1102,5678,7719,...,49,51,0,5,40,45,0,3,48,51
984,E02000983,Westminster 024,8434,1257,1749,2300,1834,1294,5883,9032,...,29,34,0,4,38,42,0,2,41,43
